Created By Muhammad Hanan Asghar

My Github: https://github.com/MuhammadHananAsghar

**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Mon Dec 14 07:54:20 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
Kaggle	yolov3


**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14621, done.
remote: Total 14621 (delta 0), reused 0 (delta 0), pack-reused 14621
Receiving objects: 100% (14621/14621), 13.21 MiB | 24.64 MiB/s, done.
Resolving deltas: 100% (9955/9955), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=21@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-12-14 07:58:33--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   588KB/s    in 4m 32s  

2020-12-14 08:03:06 (583 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/datasets/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.456000 0.498000 0.720000 0.756000

['0', '0.456000', '0.498000', '0.720000', '0.756000']
0 data/obj/datasets/295736555.txt
0 0.456000 0.498000 0.720000 0.756000
0 0.440000 0.510000 0.696000 0.748000

['0', '0.440000', '0.510000', '0.696000', '0.748000']
1 data/obj/datasets/95822204.txt
0 0.440000 0.510000 0.696000 0.748000
1 0.502000 0.624000 0.460000 0.376000

['1', '0.502000', '0.624000', '0.460000', '0.376000']
2 data/obj/datasets/344975707.txt
0 0.502000 0.624000 0.460000 0.376000
0 0.442000 0.514000 0.692000 0.724000

['0', '0.442000', '0.514000', '0.692000', '0.724000']
3 data/obj/datasets/330014533.txt
0 0.442000 0.514000 0.692000 0.724000
0 0.432000 0.522000 0.688000 0.724000

['0', '0.432000', '0.522000', '0.688000', '0.724000']
4 data/obj/datasets/305076326.txt
0 0.432000 0.522000 0.688000 0.724000
0 0.480000 0.480000 0.696000 0.728000

['0', '0.480000', '0.480000', '0.696000', '0.728000']
5 data/obj/datasets/318685212.txt
0 0.480000 0.480000 0.696000 0.728000
0 0.500000 

In [ ]:
import glob
images_list = glob.glob("data/obj/datasets/*.jpg")
print(images_list)

['data/obj/datasets/309162021.jpg', 'data/obj/datasets/142671020.jpg', 'data/obj/datasets/83263101.jpg', 'data/obj/datasets/163257365.jpg', 'data/obj/datasets/233088752.jpg', 'data/obj/datasets/390953004.jpg', 'data/obj/datasets/255529255.jpg', 'data/obj/datasets/13686601.jpg', 'data/obj/datasets/140867242.jpg', 'data/obj/datasets/157573755.jpg', 'data/obj/datasets/48177760.jpg', 'data/obj/datasets/134302549.jpg', 'data/obj/datasets/192842044.jpg', 'data/obj/datasets/180691715.jpg', 'data/obj/datasets/262982116.jpg', 'data/obj/datasets/37861431.jpg', 'data/obj/datasets/95822204.jpg', 'data/obj/datasets/295736555.jpg', 'data/obj/datasets/64363022.jpg', 'data/obj/datasets/79246788.jpg', 'data/obj/datasets/84819931.jpg', 'data/obj/datasets/263012591.jpg', 'data/obj/datasets/60946556.jpg', 'data/obj/datasets/379292803.jpg', 'data/obj/datasets/115113452.jpg', 'data/obj/datasets/68997648.jpg', 'data/obj/datasets/165666816.jpg', 'data/obj/datasets/190080001.jpg', 'data/obj/datasets/20742821.j

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
# !./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

In [ ]:
# I trained almost 3 hours